In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import os 

from skimage import io, transform
from PIL import Image as im
import cv2

from sklearn import datasets, svm, metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate

from __future__ import print_function
import keras
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from sklearn.metrics import classification_report

In [2]:
# not using this! 
def load_images(folder):
    images = [] 
    labels = []
    for filename in os.listdir(folder):
#         print(filename)
        image = io.imread(os.path.join(folder, filename))
        if image is not None:
            if filename[-5] == 'R':
                images.append(image)
#                 images.append(image)
                labels.append(filename[-6])
            # display_image(image)
    images =  np.asarray(images)
    labels = np.asarray(labels)
    return images, labels

In [3]:
# Loading the images using cv2. This will give us the right dimenstion (5, 32, 32, 3)
def load(folder):
    images = [] 
    labels = [] 
    for filename in os.listdir(folder):
        image = cv2.imread(os.path.join(folder, filename))
        if image is not None:
            if filename[-5] == 'L':
                image = cv2.resize(image, (32,32))
                images.append(image)
                labels.append(filename[-6])
    images =  np.asarray(images)
    labels = np.asarray(labels)
    return images, labels

In [4]:
def display_image(img):
    plt.figure()
    plt.imshow(img) 
    plt.show()  

In [ ]:
# X_train, X_test = load_images('./try/')
# X_train.shape

In [5]:
# obtain the training and testing dataset from the data folder
X_train, Y_train = load('./data/train/')
X_test, Y_test = load('./data/test/')

print("X training dataset: ", X_train.shape)
print("Y training dataset: ", Y_train.shape)
print("X testing dataset: ", X_test.shape)
print("Y testing dataset: ", Y_test.shape)

X training dataset:  (9000, 32, 32, 3)
Y training dataset:  (9000,)
X testing dataset:  (1800, 32, 32, 3)
Y testing dataset:  (1800,)


In [ ]:
X_train[444].shape
X_train[444]

In [ ]:
print(Y_train[444])
plt.imshow(X_train[444]);

In [6]:
num_classes = 6

Y_train = np_utils.to_categorical(Y_train, num_classes)
Y_test = np_utils.to_categorical(Y_test, num_classes)

In [ ]:
Y_test[0]

In [7]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
# CNN model_1 using hw5
model_1 = Sequential()

model_1.add(Conv2D(32, (5, 5), strides = (2,2), padding='same',
                 input_shape=X_train.shape[1:]))
model_1.add(Activation('relu'))

model_1.add(Conv2D(32, (5, 5), strides = (2,2)))
model_1.add(Activation('relu'))

model_1.add(MaxPooling2D(pool_size=(2, 2)))
model_1.add(Dropout(0.25))

model_1.add(Flatten())
model_1.add(Dense(32))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(num_classes))
model_1.add(Activation('softmax'))

model_1.summary()

In [ ]:
batch_size = 32

opt = keras.optimizers.RMSprop(learning_rate=0.0005, decay=1e-6)

model_1.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model_1.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=7,
              validation_data=(X_test, Y_test),
              shuffle=True)

In [ ]:
score = model_1.evaluate(X_test, Y_test)### double check

In [ ]:
prediction = model_1.predict(X_test)   ### double check, not sure

In [ ]:
y_true = np.argmax(Y_test, axis=1)     
y_pred = np.argmax(prediction, axis=1) 

In [ ]:
print(classification_report(y_true, y_pred)) 

In [ ]:
# confidence level
# model_1.predict(X_train[0:1], batch_size=None, verbose=0, steps=None)

# check what class it is? 
# Y_train[0:1]

In [ ]:
# Make a prediction with just one image. 
from tensorflow.keras.preprocessing import image
test_image = image.load_img('test_image.png', target_size = (32,32))
test_image = image.img_to_array(test_image)
test_image = test_image/255
display_image(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = model_1.predict(test_image)

In [ ]:
## confidence level for each of the classes. highest confidence = largest value
# np.around(result, decimals=1)
result

In [ ]:
## 2nd CNN architecture using VGG-16 ##

In [8]:
vgg_model = Sequential()

vgg_model.add(Conv2D(64, (5, 5), strides = (1,1), padding='same', input_shape=X_train.shape[1:]))
vgg_model.add(Activation('relu'))
vgg_model.add(Conv2D(64, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
# vgg_model.add(Dropout(0.25))  # No need to use dropout when using batch normalization 
vgg_model.add(BatchNormalization())

vgg_model.add(Conv2D(128, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(Conv2D(128, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
# vgg_model.add(Dropout(0.25))
vgg_model.add(BatchNormalization())

vgg_model.add(Conv2D(256, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(Conv2D(256, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
# vgg_model.add(Dropout(0.25))
vgg_model.add(BatchNormalization())

vgg_model.add(Conv2D(512, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(Conv2D(512, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(Conv2D(512, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
# vgg_model.add(Dropout(0.25))
vgg_model.add(BatchNormalization())

vgg_model.add(Conv2D(512, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(Conv2D(512, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(Conv2D(512, (3, 3), strides = (1,1), padding='same'))
vgg_model.add(Activation('relu'))
vgg_model.add(MaxPooling2D(pool_size=(2, 2)))
# # vgg_model.add(Dropout(0.5))
vgg_model.add(BatchNormalization())

vgg_model.add(Flatten())
vgg_model.add(Dense(512))
vgg_model.add(Activation('relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(Dense(num_classes))

vgg_model.add(Dense(4096))
vgg_model.add(Activation('relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(Dense(num_classes))

vgg_model.add(Dense(1000))
vgg_model.add(Activation('relu'))
vgg_model.add(Dropout(0.5))
vgg_model.add(Dense(num_classes))

vgg_model.add(Activation('softmax'))

vgg_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        4864      
_________________________________________________________________
activation (Activation)      (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 16, 16, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       7

In [9]:
opt = keras.optimizers.RMSprop(learning_rate=0.0005, decay=1e-6)

vgg_model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

vgg_model.fit(X_train, Y_train,
              batch_size=32,
              epochs=2,
              validation_data=(X_test, Y_test),
              shuffle=True)

Epoch 1/2
282/282 [==============================] - 148s 519ms/step - loss: 1.0088 - accuracy: 0.5508 - val_loss: 5.5845 - val_accuracy: 0.1667
Epoch 2/2
282/282 [==============================] - 142s 504ms/step - loss: 0.3355 - accuracy: 0.8898 - val_loss: 0.3902 - val_accuracy: 0.8211


In [ ]:
'''
References 
 - https://github.com/evernext10/Hand-Gesture-Recognition-Machine-Learning
 - https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model

'''